![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
# Face Recognition & Tracking Tutorials
This notebook has two tutorials:
1. How to perform face recognition tasks step by step
2. How to perform real-time face tracking

Prerequisites (uncomment to execute):

In [ ]:
# install degirum-face package
# %pip install degirum_face

# create cloud API access token
# !degirum token create

### Face Recognition Tutorial

Define all necessary objects:
- face embeddings database
- face recognition configuration
- create face recognition object

In [ ]:
import degirum_face, degirum_tools, numpy as np

hardware_to_use = "N2X/ORCA1"  # hardware to use for model inference

# create face embeddings database control object
db = degirum_face.ReID_Database("temp/tutorial_db.lance")  # path to the database files

# define model specs to use
face_detection_model = (
    degirum_face.model_registry.for_task("face_detection")
    .for_hardware(hardware_to_use)
    .top_model_spec()
)
face_embedding_model = (
    degirum_face.model_registry.for_task("face_embedding")
    .for_hardware(hardware_to_use)
    .top_model_spec()
)

# define face recognition configuration
face_recognition_config = degirum_face.FaceRecognitionConfig(
    face_detection_model=face_detection_model,
    face_embedding_model=face_embedding_model,
    db=db,
)

# create FaceRecognition instance: this is the main object to use for face recognition
face_recognition = degirum_face.FaceRecognition(face_recognition_config)

Define helper function to display images and videos in Jupyter notebooks
(this is just a set of service functions for convenience)

In [ ]:
from IPython.display import Image, Video, display
import cv2, tempfile


def image_display(img):
    display(Image(data=cv2.imencode(".png", img)[1].tobytes()))


def video_display(video_bytes):
    with tempfile.TemporaryDirectory() as tmpdir:
        video_path = f"{tmpdir}/video.mp4"
        with open(video_path, "wb") as f:
            f.write(video_bytes)

        display(Video(filename=video_path, width=640, height=480, embed=True))

Enroll persons in the database by analyzing their face images to extract and store face embeddings

In [ ]:
# clear all tables to start fresh
db.clear_all_tables()

# enroll Alice and Bob
enrolled = face_recognition.enroll_batch(
    ("assets/Alice-1.png", "assets/Bob-1.png"), ("Alice", "Bob")
)

# show number of enrolled embeddings in database
print(db.count_embeddings())

Recognize enrolled persons using their other images

In [ ]:
# loop over other images
recognized = []
for result in face_recognition.recognize_batch(
    (
        "assets/Alice-2.png",
        "assets/Alice-3.png",
        "assets/Bob-2.png",
        "assets/Bob-3.png",
        "assets/Alice&Bob.png",
    )
):
    # print recognition results
    print(f"\nImage '{result.info}'---------------")
    for n, face in enumerate(result.results):
        face_result = degirum_face.FaceRecognitionResult.from_dict(face)
        print(f"\nFace #{n}:\n{face_result}")
        recognized.append(face_result.embeddings[0])

    # display image overlay
    image_display(result.image_overlay)

Compute pairwise cosine similarities over face embeddings (just for demo purposes)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between every pair (recognized, enrolled)
sim_matrix = cosine_similarity(np.array(recognized), np.array(enrolled))
print("Cosine similarity matrix\n [Alice      Bob       ]")
print(sim_matrix)

### Face Tracking Tutorial

Define face tracking configuration

In [ ]:
# define configuration for face filtering
face_filter_config = degirum_face.FaceFilterConfig(
    enable_small_face_filter=True,
    min_face_size=30,
    enable_zone_filter=True,
    zone=[[100, 10], [960, 10], [960, 700], [100, 700]],
    enable_frontal_filter=True,
    enable_shift_filter=True,
    enable_reid_expiration_filter=True,
    reid_expiration_frames=10,
)

# define configuration for clip storage (use local directory for tutorial, but can be any S3-compatible storage)
clip_storage_config = degirum_tools.ObjectStorageConfig(
    endpoint="./temp", access_key="", secret_key="", bucket="tutorial_videos"
)

# define face tracking configuration
face_tracking_config = degirum_face.FaceTrackingConfig(
    video_source="assets/WalkingPeople.mp4",
    face_detection_model=face_detection_model,
    face_embedding_model=face_embedding_model,
    db=db,
    face_filter_config=face_filter_config,
    clip_storage_config=clip_storage_config,
    clip_duration=300,    
    alert_mode=degirum_face.AlertMode.ON_UNKNOWNS,
    credence_count=3,
)

# create clip manager instance
clip_manager = degirum_face.FaceClipManager(face_tracking_config)

Run pipeline on database. We will collect video clips of unknown persons.

In [ ]:
# clear all saved clips first
clip_manager.remove_all_clips()

# then run the face tracking pipeline
composition, _ = degirum_face.start_face_tracking_pipeline(face_tracking_config)
composition.wait()

Analyze collected video clips

In [ ]:
# list all collected clips
all_clips = clip_manager.list_clips()
print(f"Clips saved: {len(all_clips)}")

# show them by downloading from storage:
for clip in all_clips:
    video_display(clip_manager.download_clip(clip + ".mp4"))

Annotate collected video clips

In [ ]:
for clip in all_clips:
    # annotate clip: it will return face map object indexed by object track IDs and containing face embeddings
    face_map = clip_manager.find_faces_in_clip(clip)

    # show annotated clip
    video_display(
        clip_manager.download_clip(clip + clip_manager.annotated_video_suffix)
    )

    break  # annotate only first clip

Add (enroll) all persons detected on the video clip to the database

In [ ]:
# You can see on a video above that object #1 is Bob and object #2 is Alice

# Add embeddings for each object to the database
face_tracking_config.db.add_embeddings_for_attributes("Bob", face_map.map[1].embeddings)
face_tracking_config.db.add_embeddings_for_attributes("Alice", face_map.map[2].embeddings)

# show number of enrolled embeddings in database
print(face_tracking_config.db.count_embeddings())

Run pipeline one more time: now should be no alerts

In [ ]:
# clear all saved clips first
clip_manager.remove_all_clips()

# start face tracking pipeline
composition, _ = degirum_face.start_face_tracking_pipeline(face_tracking_config)
composition.wait()

clips = clip_manager.list_clips()
print(f"Now we have: {len(clips)} clip(s)")
if not clips:
    print("No alerts detected!")